In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer

## Meshcat for Visualization

In [2]:
meshcat = StartMeshcat()

# When this notebook is run in test mode it needs to stop execution without
# user interaction. For interactive model visualization you won't normally
# need the 'loop_once' flag.
test_mode = True if "TEST_SRCDIR" in os.environ else False

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Initial setup 

The following scripts look for a directorty containing directories of google scanned object data and then generates the necessary files and randomly initializes positions within a specified radius of the origin. 

To add more objects to the scene, simply download the zip files from [Google Scanned Objects](https://app.gazebosim.org/GoogleResearch/fuel/collections/Scanned%20Objects%20by%20Google%20Research). Then run the following bash script from within the directory containing only the zip files. 

**Note 1:** that you'll ultimately place `~/tmp/OBJECTS` in the `IMPACT` directory. 

**Note 2:** this script assumes that the `IMPACT` directory is in the user's home directory. Modify accordingly. 



```bash
#!/usr/bin/env bash
mkdir ~/tmp
mkdir ~/tmp/OBJECTS
echo name,sdf_file,mass_kg,x_init,y_init,z_init,roll_init,pitch_init,yaw_init >> ~/tmp/object_manifest.csv
for i in $(ls zippers|sed 's/.zip//'); do  
    mkdir ~/tmp/OBJECTS/$i; 
    unzip -d ~/tmp/OBJECTS/$i zippers/$i.zip; 
    echo "$i","$HOME/IMPACT/OBJECTS/$i/model.sdf",mass,x,y,z,roll,pitch,yaw >> ~/tmp/object_manifest.csv; 
    cp ~/tmp/OBJECTS/$i/materials/textures/texture.png ~/tmp/OBJECTS/$i/meshes; 
done
```

### Place objects
Given a directory of object directories and a clutter radius randomly place the objects and set their masses

```python
def update_object_manifest(csv_file, radius):
```

In [3]:
import pandas as pd
import numpy as np
import os

MANIFEST_PATH = "manifests"  # Replace with the actual path to the manifests

def update_object_manifest(csv_file, radius):
    # Read the object manifest CSV file
    df = pd.read_csv(csv_file)
    
    # Generate random initial positions within the specified radius
    num_objects = len(df)
    x_init = np.random.uniform(low=-radius, high=radius, size=num_objects)
    y_init = np.random.uniform(low=-radius, high=radius, size=num_objects)
    z_init = np.zeros(num_objects)
    
    # Generate random initial orientations
    roll_init = np.random.uniform(low=-np.pi, high=np.pi, size=num_objects)
    pitch_init = np.random.uniform(low=-np.pi, high=np.pi, size=num_objects)
    yaw_init = np.random.uniform(low=-np.pi, high=np.pi, size=num_objects)
    
    # Generate random mass values between 0.5 and 10 with 0.1 kg increments
    mass_kg = np.arange(0.5, 10.1, 0.1)
    mass_kg = np.random.choice(mass_kg, size=num_objects)
    
    # Update the dataframe with the new values
    df['x_init'] = x_init
    df['y_init'] = y_init
    df['z_init'] = z_init
    df['roll_init'] = roll_init
    df['pitch_init'] = pitch_init
    df['yaw_init'] = yaw_init
    df['mass_kg'] = mass_kg
    
    largest_manifest = -1
    manifest_files = [filename for filename in os.listdir(MANIFEST_PATH) if filename.endswith("_object_manifest.csv")]
    if manifest_files:
        largest_manifest = max([int(filename.split("_")[0]) for filename in manifest_files])

    new_manifest_number = largest_manifest + 1
    new_csv_file = os.path.join(MANIFEST_PATH, f"{str(new_manifest_number).zfill(2)}_object_manifest.csv")

    # Save the updated dataframe to the new CSV file
    df.to_csv(new_csv_file, index=False)

    new_csv_file
    largest_manifest = max([int(filename.split("_")[0]) for filename in os.listdir(MANIFEST_PATH) if filename.endswith("_object_manifest.csv")])
    new_manifest_number = largest_manifest + 1
    new_csv_file = os.path.join(MANIFEST_PATH, f"{str(new_manifest_number).zfill(2)}_object_manifest.csv")
    
    # Save the updated dataframe to the new CSV file
    df.to_csv(new_csv_file, index=False)
    
    return new_csv_file


### Generate SDF files
Given a manifest of objects, generate a SDF file for each object.

```python
def replace_sdf_files(manifest_csv):
```

In [4]:
SDF_TEMPLATE = """<sdf version='1.6'>
  <model name='{OBJECT}'>
    <link name='{OBJECT}_link'>
      <!-- Inertial properties -->
      <!-- Mass of the {OBJECT} in kilograms -->
      <inertial>
        <mass>{OBJ_MASS}</mass> <!-- Approximate mass -->
        <!-- Inertia tensor components; these are estimated values for a roughly symmetric object -->
        <!-- The actual values depend on the specific distribution of mass in the {OBJECT} -->
        <inertia>
          <ixx>0.0002</ixx> <!-- Moment of inertia about the X-axis -->
          <iyy>0.0002</iyy> <!-- Moment of inertia about the Y-axis -->
          <izz>0.0002</izz> <!-- Moment of inertia about the Z-axis -->
          <ixy>0</ixy>      <!-- Product of inertia (likely zero for a symmetric object) -->
          <ixz>0</ixz>
          <iyz>0</iyz>
        </inertia>
      </inertial>

      <!-- Visual properties (appearance of the teapot) -->
      <visual name='teapot_visual'>
        <geometry>
          <mesh>
            <uri>file:///home/jarret/IMPACT/OBJECTS/{OBJECT}/meshes/model.obj</uri> <!-- Path to the OBJ file -->
            <scale>1 1 1</scale> <!-- Adjust scale if necessary -->
          </mesh>
        </geometry>
        <!-- Optional material properties like color, texture -->
      </visual>

      <!-- Collision properties -->
      <!-- Using the same geometry for collision; consider simplifying for complex shapes -->
      <collision name='{OBJECT}_collision'>
        <geometry>
          <mesh>
            <uri>file:///home/jarret/IMPACT/OBJECTS/{OBJECT}/meshes/model.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

In [5]:
import os

def replace_sdf_files(manifest_csv):
    # Read the object manifest CSV file
    df = pd.read_csv(manifest_csv)
    
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Get the object name and mass from the manifest
        object_name = row['name']
        obj_mass = row['mass_kg']
        
        # Get the current SDF file path
        sdf_file = row['sdf_file']
        
        # Delete the current SDF file if it exists
        if os.path.exists(sdf_file):
            os.remove(sdf_file)
        
        # Generate the SDF content by replacing the placeholders in the SDF_TEMPLATE string
        sdf_content = SDF_TEMPLATE.format(OBJECT=object_name, OBJ_MASS=obj_mass)
        
        # Write the SDF content to a new file
        with open(sdf_file, 'w') as file:
            file.write(sdf_content)

## Creating the Scene from a Manifest

```python
def generate_scene_from_manifest(manifest_csv, sim_time_step):
```

Given a manifest file, generate the corresponding scene graph

In [6]:
def generate_scene_from_manifest(manifest_csv, sim_time_step):
    # Create MultibodyPlant and SceneGraph
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    
    # Create Parser
    parser = Parser(plant)
    
    # Read the object manifest CSV file
    df = pd.read_csv(manifest_csv)
    
    # Randomly select 20 objects from the manifest
    selected_objects = df.sample(n=min(10, len(df)))

    # Create a KUKA iiwa arm
    # iiwa7_model_url = (
    #     "package://drake/manipulation/models/"
    #     "iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
    # parser.AddModels(url=iiwa7_model_url)
    
    for index, row in selected_objects.iterrows():
        # Get the SDF file path from the manifest
        sdf_file = row['sdf_file']
        
        # Add the SDF file to the parser
        parser.AddModels(sdf_file)
        
        # Get the initial position from the manifest
        x_init = row['x_init']
        y_init = row['y_init']
        z_init = row['z_init']
        roll_init = row['roll_init']
        pitch_init = row['pitch_init']
        yaw_init = row['yaw_init']
        
        # Create a RigidTransform for the initial position
        X_WO = RigidTransform(
            RollPitchYaw(roll_init, pitch_init, yaw_init),
            [x_init, y_init, z_init]
        )
        
        # Add the object to the MultibodyPlant
        plant.WeldFrames(
            plant.world_frame(),
            plant.GetFrameByName(f"{row['name']}_link"),
            X_WO
        )
    
    plant.Finalize()

    # Add visualization to see the geometries.
    AddDefaultVisualization(builder=builder, meshcat=meshcat)
    
    # Build the diagram
    diagram = builder.Build()
    
    return diagram


## Action

In [7]:
def initialize_simulation(diagram):
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    return simulator

def run_simulation(manifest, sim_time_step):
    diagram = generate_scene_from_manifest(manifest, sim_time_step)
    simulator = initialize_simulation(diagram)
    meshcat.StartRecording()
    finish_time = 0.1 if test_mode else 2.0
    simulator.AdvanceTo(finish_time)
    meshcat.PublishRecording()

In [9]:
manifester = update_object_manifest("object_manifest_template.csv", .5)
replace_sdf_files(manifester)

# Run the simulation with a small time step. Try gradually increasing it!
run_simulation(manifester,sim_time_step=0.0001)

[Visualization](http://localhost:7000)